In [ ]:
import os

class FileLoader:
    def __init__(self, filename, columns, filetype):
        self.filename = filename
        self.columns = columns
        self.filetype = filetype
        self.file = None
        self.open(filename, columns, filetype)

    def open(self, filename, columns, filetype):
        mode = 'r' if filetype == 'TEXT' else 'rb'
        try:
            self.file = open(filename, mode)
            return True
        except IOError:
            print("File could not be opened")
            return False

    def load(self):
        return self.load_()

    def loadn(self, epochs):
        datas = []
        for _ in range(epochs):
            data = self.load_()
            if data is not None:
                datas.append(data)
            else:
                break
        return datas

    def load_single(self):
        data = self.load_()
        if data is not None:
            return data
        else:
            return None

    def load_multiple(self, epochs):
        datas = []
        for _ in range(epochs):
            data = self.load_()
            if data is not None:
                datas.append(data)
            else:
                break
        return datas

    def load_(self):
        if self.is_eof():
            return None

        if self.filetype == 'TEXT':
            line = self.file.readline()
            if not line:
                return None
            # Skipping whitespace and splitting by any combination of comma and whitespace
            splits = [x.strip() for x in line.split(",") if x.strip()]
            data = [float(val) for val in splits]
            if len(data) != self.columns:
                raise ValueError("Number of columns does not match data")
            return data
        else:
            import struct
            data_bytes = self.file.read(self.columns * 8)  # assuming double precision
            if not data_bytes:
                return None
            data = struct.unpack('d' * self.columns, data_bytes)
            return list(data)

    def is_eof(self):
        # Checks if we are at the end of the file
        current_position = self.file.tell()
        self.file.seek(0, os.SEEK_END)
        if current_position == self.file.tell():
            return True
        self.file.seek(current_position)
        return False

# Constants for file types
TEXT = 'TEXT'
BINARY = 'BINARY'

# Example usage
file_loader = FileLoader('data.txt', 5, TEXT)
data = file_loader.load_multiple(10)
print(data)
